In [1]:
import os
import pandas as pd
import glob

#現在のディレクトリを確認
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: /Users/sawad/dev/myapps/u-on-zu_data-processing


In [2]:
# 読み込み元ディレクトリを指定
input_dir = './raw/utf-8'

#出力先ディレクトリを指定
output_dir = './processed'

In [3]:

# ディレクトリ内のすべてのCSVファイルのファイル名を取得
csv_files = glob.glob(os.path.join(input_dir, "*.csv"))

# ファイル名のみを抽出
csv_filenames = [os.path.basename(file) for file in csv_files]

csv_filenames.sort()

# ファイル名を表示
print("ディレクトリ内のCSVファイル一覧:")
for filename in csv_filenames:
    print(filename)

# # CSVファイルの読み込みテスト
# for file in csv_files:
#     # ファイルを読み込み
#     df = pd.read_csv(file, encoding='utf-8')
#     display(df.head(6))
#     df.to_csv(os.path.join(output_dir, 'new_df.csv'), index=False, header=False)


ディレクトリ内のCSVファイル一覧:
11_宗谷_utf-8.csv
12_上川_utf-8.csv
13_留萌_utf-8.csv
14_石狩_utf-8.csv
15_空知_utf-8.csv
16_後志_utf-8.csv
17_オホーツク_utf-8.csv
18_根室_utf-8.csv
19_釧路_utf-8.csv
20_十勝_utf-8.csv
21_胆振_utf-8.csv
22_日高_utf-8.csv
23_渡島_utf-8.csv
24_檜山_utf-8.csv
31_青森_utf-8.csv
32_秋田_utf-8.csv
33_岩手_utf-8.csv
34_宮城_utf-8.csv
35_山形_utf-8.csv
36_福島_utf-8.csv
40_茨城_utf-8.csv
41_栃木_utf-8.csv
42_群馬_utf-8.csv
43_埼玉_utf-8.csv
44_東京_utf-8.csv
45_千葉_utf-8.csv
46_神奈川_utf-8.csv
48_長野_utf-8.csv
49_山梨_utf-8.csv
50_静岡_utf-8.csv
51_愛知_utf-8.csv
52_岐阜_utf-8.csv
53_三重_utf-8.csv
54_新潟_utf-8.csv
55_富山_utf-8.csv
56_石川_utf-8.csv
57_福井_utf-8.csv
60_滋賀_utf-8.csv
61_京都_utf-8.csv
62_大阪_utf-8.csv
63_兵庫_utf-8.csv
64_奈良_utf-8.csv
65_和歌山_utf-8.csv
66_岡山_utf-8.csv
67_広島_utf-8.csv
68_島根_utf-8.csv
69_鳥取_utf-8.csv
71_徳島_utf-8.csv
72_香川_utf-8.csv
73_愛媛_utf-8.csv
74_高知_utf-8.csv
81_山口_utf-8.csv
82_福岡_utf-8.csv
83_大分_utf-8.csv
84_長崎_utf-8.csv
85_佐賀_utf-8.csv
86_熊本_utf-8.csv
87_宮崎_utf-8.csv
88_鹿児島_utf-8.csv
91_沖縄_utf-8.csv


In [4]:
# climate_data_test.py を元に,各ファイルを整形する関数を定義
def shape_city_data(df):
    #6行目（ヘッダーなし）に「平年値」の文字列が含まれている行を抽出 （columns_with_heinenchiに条件式の結果がTとなるindexの配列が返る）
    columns_with_heinenchi = df.columns[df.iloc[5].str.contains('平年値', na=False)]
    df_heinenchi = df[columns_with_heinenchi]
    
    # 7行目（ヘッダーなし）がNaNのカラム（=データが入っているカラム）について，8〜19行目に1つでも空欄が含まれていれば，4行目の観測地点名を配列にpushする
    # 除外都市リストを宣言
    exclude_cities = []
    
    # 7行目がNaNのカラム（=実際のデータが入っているカラム）を抽出
    columns_onlydata = df_heinenchi.columns[df_heinenchi.iloc[6].isna()]
    df_onlydata = df[columns_onlydata]
    
    # 実データ部分のみをsubset化
    data_subset = df_onlydata.iloc[7:18]
    # 実データ部分のいずれか1つでもNaNのあるカラムのindexを特定
    columns_with_nan = data_subset.columns[data_subset.isna().any()]    
    
    # 平年値データに戻り,実データにNaNを含むカラムの都市名を配列にpush
    for col in columns_with_nan:
        exclude_cities.append(df_onlydata.loc[3, col])
    
    # 「品質情報」のカラムを抜き出し,8〜19行目に1つでも「8」以外の数値が含まれていれば，4行目の観測地点名を配列にpushする
    columns_onlyquality = df_heinenchi.columns[df_heinenchi.iloc[6].str.contains('品質情報', na=False)]
    df_onlyquality = df[columns_onlyquality]
    
    # 品質情報の実データ部分のみを抽出
    data_quality_subset = df_onlyquality.iloc[7:18]
    # display(data_quality_subset)
    
    # 実データ部分のいずれか1つでも8以外のあるカラムのindexを特定
    data_quality_subset = data_quality_subset.astype(int)  # int型に変換
    columns_with_notok = data_quality_subset.columns[(data_quality_subset != 8).any()]
    # display(columns_with_notok)
    
    # 平年値データに戻り,8以外のあるカラムの都市名を配列にpush
    for col in columns_with_notok:
        exclude_cities.append(df_onlyquality.loc[3, col])
        
    # 除外都市リストの重複を除外しておく
    exclude_cities = list(set(exclude_cities))

    
    # 除外都市リストに含まれている都市のデータを,気温・降水量共に除外する
    # データフレームから都市名の行をインデックスと共に抽出
    city_name = df_onlydata.loc[3]
    
    # exclude_citiesに名前が登録されている都市の,データフレームのカラムindexを配列で取得
    exclude_indexes = city_name[city_name.isin(exclude_cities)].index
    
    # 除外するカラムindexでフィルタリング
    filtered_df = df_onlydata.drop(columns=exclude_indexes)
    
    # 仕上げ
    # 1,2,7行目を削除
    final_df = filtered_df.drop([0,1,6])
    return final_df

In [20]:
# テストコード
df=pd.read_csv('./raw/utf-8/18_根室_utf-8.csv', encoding='utf-8', header=None)
# display(df)
shaped_data = shape_city_data(df)

display(shaped_data)
#繰り返し処理で,2列のデータをまとめて最終出力用データフレームにappendする
# 列を2つずつ繰り返し処理
for i in range(0, shaped_df.shape[1], 2):      #.shape[1]で列数を返す
    
    # 2列にわたるデータから,都府県名,観測所名,temp,rainデータを抜き出して1つの配列にまとめる
    area_and_name = shaped_df.iloc[0:2, i]

    # 気温と降水量がどちらが先でもよいように対応
    label = shaped_df.iloc[2, i]
    if "平均気温" in label:
        temp = shaped_df.iloc[4:16, i]    
        rain = shaped_df.iloc[4:16, i+1]
    else:
        rain = shaped_df.iloc[4:16, i]
        temp = shaped_df.iloc[4:16, i+1]

    new_row = pd.concat([area_and_name, temp, rain])
    display(new_row)


,4,9,14,19,24,29,34,39,54,59,65,71,76,81,86,91
2,根室,根室,根室,根室,根室,根室,根室,根室,根室,根室,根室,根室,根室,根室,根室,根室
3,羅臼,羅臼,標津,標津,上標津,上標津,中標津,中標津,別海,別海,根室,根室,納沙布,納沙布,厚床,厚床
4,降水量の合計(mm),平均気温(℃),降水量の合計(mm),平均気温(℃),降水量の合計(mm),平均気温(℃),降水量の合計(mm),平均気温(℃),降水量の合計(mm),平均気温(℃),降水量の合計(mm),平均気温(℃),降水量の合計(mm),平均気温(℃),降水量の合計(mm),平均気温(℃)
5,平年値(mm),平年値(℃),平年値(mm),平年値(℃),平年値(mm),平年値(℃),平年値(mm),平年値(℃),平年値(mm),平年値(℃),平年値(mm),平年値(℃),平年値(mm),平年値(℃),平年値(mm),平年値(℃)
7,89.7,-4.5,38.0,-5.4,45.1,-6.7,44.0,-6.6,38.2,-6.7,30.6,-3.4,18.9,-3.0,37.2,-6.0
8,66.6,-4.8,28.0,-5.5,37.9,-6.4,32.0,-6.5,27.4,-6.5,23.5,-3.8,14.6,-3.7,27.7,-5.8
9,93.9,-1.1,54.3,-1.6,57.8,-2.1,57.5,-2.0,55.2,-1.9,47.0,-0.8,37.4,-1.0,57.8,-1.7
10,111.7,2.9,82.8,3.4,85.4,3.0,84.2,3.6,79.0,3.6,64.4,3.5,53.9,3.0,78.8,3.3
11,144.0,7.8,109.8,8.1,108.6,8.8,107.5,8.9,107.2,8.7,96.2,7.7,82.9,6.8,110.2,7.9
12,119.4,11.5,111.9,11.5,107.6,12.8,103.9,12.6,111.4,12.3,103.0,10.9,93.6,10.1,117.1,11.5


2        檜山
3       せたな
7      -1.9
8      -1.4
9       1.7
10      6.5
11     11.1
12     15.3
13     19.5
14     21.4
15     18.2
16     12.4
17      6.1
18      0.3
7      53.7
8      48.3
9      51.1
10     68.8
11     94.9
12     67.3
13    116.3
14    145.5
15    145.4
16    104.6
17     99.1
18     72.0
dtype: object

2        檜山
3        今金
7      -3.6
8      -3.0
9       0.3
10      5.7
11     11.1
12     15.4
13     19.5
14     21.1
15     17.1
16     10.6
17      4.3
18     -1.4
7     105.0
8      85.8
9      71.7
10     78.6
11    105.5
12     74.4
13    131.4
14    166.0
15    151.4
16    126.3
17    124.3
18    121.7
dtype: object

2        檜山
3        奥尻
7      -0.4
8      -0.2
9       3.0
10      7.4
11     11.7
12     16.0
13     20.1
14     22.5
15     20.0
16     14.2
17      7.6
18      1.7
7      29.6
8      34.1
9      42.4
10     64.2
11     83.8
12     76.3
13    113.3
14    127.1
15    122.7
16     90.4
17     84.2
18     61.9
dtype: object

2        檜山
3        米岡
7      -1.6
8      -1.0
9       2.3
10      6.8
11     11.3
12     15.8
13     19.8
14     22.2
15     19.4
16     13.5
17      7.0
18      0.8
7      31.3
8      30.9
9      40.3
10     53.0
11     87.4
12     82.7
13    108.4
14    112.0
15    113.8
16     76.8
17     74.1
18     57.7
dtype: object

2        檜山
3         鶉
7      -3.7
8      -3.0
9       0.5
10      6.0
11     11.4
12     15.8
13     19.8
14     21.2
15     17.1
16     10.5
17      4.5
18     -1.3
7      91.3
8      73.2
9      72.0
10     80.9
11    103.5
12     86.2
13    143.7
14    186.5
15    149.7
16    122.1
17    131.3
18    112.7
dtype: object

2        檜山
3        江差
7      -0.6
8      -0.2
9       2.9
10      7.7
11     12.3
12     16.4
13     20.6
14     22.6
15     19.5
16     13.7
17      7.5
18      1.6
7      84.9
8      68.4
9      63.6
10     74.7
11     98.0
12     78.6
13    126.4
14    167.0
15    135.6
16    105.5
17    118.6
18    109.5
dtype: object

NameError: name '平年値' is not defined

In [5]:



# 最終出力用の新しいデータフレームを作成
new_df = pd.DataFrame(columns=range(0,26))
# ヘッダーを追加
header = ['都府県振興局', '観測所名', 
          'temp_jan', 'temp_feb', 'temp_mar', 'temp_apr', 'temp_may', 'temp_jun',
          'temp_jul', 'temp_aug', 'temp_sep', 'temp_oct', 'temp_nov', 'temp_dec',
          'rain_jan', 'rain_feb', 'rain_mar', 'rain_apr', 'rain_may', 'rain_jun',
          'rain_jul', 'rain_aug', 'rain_sep', 'rain_oct', 'rain_nov', 'rain_dec']
new_df.loc[0] = header

# ディレクトリ内のすべてのCSVファイルのファイル名を取得
csv_files = glob.glob(os.path.join(input_dir, "*.csv"))


# # csv_filesで繰り返し処理
for file in csv_files:
    display(file)
    # ファイルを読み込み
    df = pd.read_csv(file, encoding='utf-8', header=None)
    # display(df.head(6))

    # 上記で定義した整形関数を実行し,完成したdfを返り値で受ける
    shaped_df = shape_city_data(df)

    #繰り返し処理で,2列のデータをまとめて最終出力用データフレームにappendする
    # 列を2つずつ繰り返し処理
    for i in range(0, shaped_df.shape[1], 2):      #.shape[1]で列数を返す
        
        # 2列にわたるデータから,都府県名,観測所名,temp,rainデータを抜き出して1つの配列にまとめる
        area_and_name = shaped_df.iloc[0:2, i]

        # 気温と降水量がどちらが先でもよいように対応
        label = shaped_df.iloc[2, i]
        if "平均気温" in label:
            temp = shaped_df.iloc[4:16, i]    
            rain = shaped_df.iloc[4:16, i+1]
        else:
            rain = shaped_df.iloc[4:16, i]
            temp = shaped_df.iloc[4:16, i+1]
    
        new_row = pd.concat([area_and_name, temp, rain])
        # display(new_row)
        
        # 作成したデータ配列を,データフレームの最後尾に格納（len(new_df)で既存配列の「長さ」を返すので,index指定で最後尾となる)
        new_df.loc[len(new_df)] = new_row.T.values   # Tで縦横変換

display(new_df)

new_df.to_csv(os.path.join(output_dir, 'new_df.csv'), index=False, header=False)

#######この後重複チェックを下で行う##############




'./raw/utf-8/56_石川_utf-8.csv'

'./raw/utf-8/71_徳島_utf-8.csv'

'./raw/utf-8/52_岐阜_utf-8.csv'

'./raw/utf-8/60_滋賀_utf-8.csv'

'./raw/utf-8/65_和歌山_utf-8.csv'

...

'./raw/utf-8/24_檜山_utf-8.csv'

'./raw/utf-8/62_大阪_utf-8.csv'

'./raw/utf-8/45_千葉_utf-8.csv'

'./raw/utf-8/63_兵庫_utf-8.csv'

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,都府県振興局,観測所名,temp_jan,temp_feb,temp_mar,temp_apr,temp_may,temp_jun,temp_jul,temp_aug,...,rain_mar,rain_apr,rain_may,rain_jun,rain_jul,rain_aug,rain_sep,rain_oct,rain_nov,rain_dec
1,石川,珠洲,2.8,2.8,5.5,10.4,15.7,19.8,24.1,25.5,...,129.2,106.6,98.4,141.7,193.9,165.7,184.9,152.7,200.5,279.2
2,石川,輪島,3.3,3.4,6.1,11.1,16.1,20.0,24.4,25.9,...,138.6,121.6,115.6,155.8,199.6,176.8,214.5,171.1,231.5,278.4
3,石川,三井,1.1,1.4,4.4,9.6,15.1,19.1,23.2,24.7,...,132.7,128.6,101.7,172.2,241.4,200.5,190.1,157.6,193.4,264.8
4,石川,志賀,3.6,3.7,6.3,11.3,16.4,20.6,24.7,26.4,...,105.7,105.2,107.6,151.9,203.8,172.5,173.9,134.7,170.9,197.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,兵庫,神戸空港,5.9,6.3,9.2,13.7,18.5,22.2,25.9,27.9,...,83.2,83.7,117.6,164.5,186.9,84.7,135.1,106.1,52.8,49.3
893,兵庫,神戸,6.2,6.5,9.8,15.0,19.8,23.4,27.1,28.6,...,94.2,100.6,134.7,176.7,187.9,103.4,157.2,118.0,62.4,48.7
894,兵庫,郡家,5.8,5.9,8.7,13.7,18.6,22.5,26.5,27.8,...,79.2,85.8,114.5,154.5,144.9,94.4,183.3,139.8,65.6,52.9
895,兵庫,洲本,5.5,5.8,9.2,14.2,19.0,22.4,26.6,27.7,...,109.0,117.5,145.0,198.2,182.2,117.0,223.6,185.6,91.5,75.1


In [6]:
# DataFrameの2列目の内容に重複があるかどうかをチェック
is_duplicate = new_df.iloc[:, 1].duplicated()

# 重複がある行を表示
duplicates = new_df[is_duplicate]
print("重複がある行:")
print(duplicates)

重複がある行:
     0   1     2     3     4     5     6     7     8     9   ...     16  \
191  福岡  八幡   6.2   6.9  10.0  14.7  19.3  22.7  26.8  27.8  ...  114.2   
288  福島  金山  -1.0  -0.8   2.1   7.3  14.3  18.5  22.6  23.6  ...  116.1   
294  福島  川内  -0.5   0.0   3.3   8.8  14.0  17.7  21.7  22.5  ...   95.1   
329  岩手  山田   0.5   0.8   3.7   8.7  13.4  16.9  20.8  22.4  ...  102.7   
363  静岡  清水   6.7   7.5  10.5  14.9  18.9  22.1  25.7  27.1  ...  209.4   
604  十勝  池田  -8.2  -6.9  -1.3   4.9  10.4  14.2  18.0  19.4  ...   40.0   
609  十勝  大津  -7.4  -6.4  -1.2   4.1   8.5  12.1  16.1  18.1  ...   47.3   
616  東京  府中   4.5   5.4   8.8  13.9  18.5  21.8  25.7  26.8  ...  113.8   
636  愛媛  長浜   6.5   6.8   9.4  13.6  17.7  21.2  25.4  26.5  ...   94.7   
654  佐賀  白石   5.0   6.2   9.6  14.5  19.3  23.1  27.0  27.6  ...  109.8   
661  群馬  田代  -4.5  -4.0  -0.4   5.7  11.2  14.9  19.1  19.6  ...   96.4   
746  山口  豊田   3.6   4.4   7.5  12.4  17.3  21.2  25.1  25.9  ...  140.5   
752  山口  下関   7.2